# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [1]:
# Install if needed
!pip install composable
!pip install composablesoup
# Check for upgrade is already installed
!pip install composable --upgrade
!pip install composablesoup --upgrade
from composable import pipeable
from composable.strict import map, filter
from composablesoup import find, find_all, get_text, has_attr
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
from composable.sequence import to_list, head
from composable.string import strip,replace, split
from composable import from_toolz as tlz

Requirement already up-to-date: composable in /home/pc7482yh/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.5)
Requirement already up-to-date: composablesoup in /home/pc7482yh/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.4)


In [2]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi')
restaurants = BeautifulSoup(r.content, 'html.parser')

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [3]:
import re
not_empty = re.compile(r'.+')
restaurant_name = (restaurants
>> find_all("a",attrs={'name': not_empty})
>> map(tlz.get('name'))
)

In [4]:
restaurant_name

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher']

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [29]:
is_ranking=re.compile(r'black-regular')
restaurant_ranking = (restaurants
                        >> find_all('span',attrs={'class':is_ranking})
                        >> map(get_text)
                     )
rank_filter = re.compile(r'\d{1,2}')
rankings = [line for line in restaurant_ranking if rank_filter.match(line)]
final_rank_name = [line.replace('\xa0'," ") for line in rankings]

In [30]:
final_rank_name

['1. The Waterfront Restaurant & Tavern',
 '2. Restore Public House',
 '3. Buzzard Billy’s',
 '4. Lovechild Restaurant',
 '5. The Charmant',
 '6. The Freighthouse Restaurant',
 '7. Howie’s on La Crosse',
 '8. Piggy’s Restaurant',
 '9. The Crow',
 '10. Schuby’s Neighborhood Butcher']

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [6]:
get_restaurant_name = (restaurants
>> find_all("a",attrs={'name': not_empty})
>> map(tlz.get('name'))
)

In [26]:
get_restaurant_name

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher']

#### Rating

In [31]:
star_rating = re.compile(r'\d+ star rating')
review_ranking = (restaurants
                    >> find_all('div',attrs={'aria-label':star_rating})
                    >> map(tlz.get('aria-label'))
                 )

In [32]:
review_ranking

['4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '3.5 star rating',
 '4.5 star rating']

#### Address

In [24]:
find_address = re.compile(r'raw_')
restaurant_address_search = (restaurants
                                 >> find_all('span',attrs={'class':find_address})
                                 >> map(get_text)
                                 )
address_filter = re.compile(r'\d{1,4}.*')
restaurant_address = [line for line in restaurant_address_search if address_filter.match(line)]

In [38]:
restaurant_address[1:]

['328 Front St S',
 '1810 State St',
 '222 Pearl St',
 '300 3rd St S',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '501 Front St S',
 '100 3rd St S',
 '321 State St']

#### Review Count

In [33]:
find_review_count = re.compile(r'reviewCount_')
restaurant_review_count = (restaurants
                            >> find_all('span',attrs={'class':find_review_count})
                            >> map(get_text)
                          )

In [34]:
restaurant_review_count

['2', '226', '17', '275', '109', '149', '125', '57', '131', '154', '17']

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [9]:
(restaurants
>>find_all("a",attrs={"role":"link"})
>> map(get_text)
)

['Yelp',
 'La Crosse',
 'Restaurants',
 'Food Trucks',
 'Mexican',
 'American (New)',
 'Seafood',
 'Steakhouses',
 'American (Traditional)',
 'American (Traditional)',
 'Cajun/Creole',
 'American (New)',
 'French',
 'Cocktail Bars',
 'Seafood',
 'Steakhouses',
 'Desserts',
 'American (New)',
 'Pubs',
 'Steakhouses',
 'Seafood',
 'Sandwiches',
 'American (Traditional)',
 'Gastropubs',
 'Butcher',
 'Delis',
 'Caterers',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '',
 'Carry Out',
 'Cheap Dinner',
 'Cheap Eats',
 'Everything',
 'Family Restaurant',
 'Local Favorite Restaurants',
 'Open Now',
 'Pickup',
 'Piggys',
 'Restaurants & Food',
 'Restaurants Downtown',
 'Restaurants Near Me',
 'Restaurants Open Late',
 'Take Out',
 'Waterfront Restaurants',
 'American (New)',
 'American (Traditional)',
 'Breakfast & Brunch',
 'Burgers',
 'Cafes',
 'Chicken Wings',
 'Chinese',
 'Delis',
 'Fast Food',
 'Hot Dogs',
 'Italian',
 'Mexican',
 'Pizza',
 'Salad',
 'Sandwiches',
 'Seafood',
 'Soup',


## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [76]:
def web_scrape(website):
    import requests
    from bs4 import BeautifulSoup
    
    s = requests.Session()
    r = s.get(website)
    site = BeautifulSoup(r.content,"html.parser")
    
    is_ranking=re.compile(r'black-regular')
    restaurant_ranking = (site
                          >> find_all('span',attrs={'class':is_ranking})
                          >> map(get_text)
                          )
    rank_filter = re.compile(r'\d{1,2}')
    rankings = [line for line in restaurant_ranking if rank_filter.match(line)]
    final_rank_name = [line.replace('\xa0'," ") for line in rankings]
    
    star_rating = re.compile(r'\d+ star rating')
    review_ranking = (site
                     >> find_all('div',attrs={'aria-label':star_rating})
                     >> map(tlz.get('aria-label'))
                     )
    review_ranking = review_ranking[1:]
    find_address = re.compile(r'raw_')
    restaurant_address_search = (site
                                 >> find_all('span',attrs={'class':find_address})
                                 >> map(get_text)
                                 )
    address_filter = re.compile(r'\d{1,4}.*')
    restaurant_address = [line for line in restaurant_address_search if address_filter.match(line)]
    restaurant_address = restaurant_address[1:]
    
    find_review_count = re.compile(r'reviewCount_')
    restaurant_review_count = (site
                               >> find_all('span',attrs={'class':find_review_count})
                               >> map(get_text)
                               )
    restaurant_review_count = restaurant_review_count[1:]
    find_category = re.compile(r'priceCategory')
    restaurant_category = (site
                    >> find_all('div', attrs={'class':find_category})
                    >> map(get_text)
                    )
    restaurant_category = restaurant_category[1:]
    category_filter = [line.replace("$","") for line in restaurant_category]
    category_semi = [line.replace(",",";") for line in category_filter]
    
    output = list(zip(final_rank_name, review_ranking, restaurant_address,restaurant_review_count, category_semi))
    joined = [",".join(line) for line in output]
    final = '\n'.join(joined)
    return final
    

In [77]:
web_scrape('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=0')

'1. The Waterfront Restaurant & Tavern,4.5 star rating,328 Front St S,226,American (New); Seafood; Steakhouses\n2. Buzzard Billy’s,4 star rating,222 Pearl St,276,American (Traditional); Cajun/Creole\n3. Restore Public House,4.5 star rating,1810 State St,17,American (Traditional)\n4. Lovechild Restaurant,4.5 star rating,300 3rd St S,109,American (New)\n5. The Charmant,4.5 star rating,101 State St,149,French; Cocktail Bars\n6. The Freighthouse Restaurant,4 star rating,107 Vine St,125,Seafood; Steakhouses; Desserts\n7. Howie’s on La Crosse,4 star rating,1128 La Crosse St,57,American (New); Pubs\n8. The Crow,3.5 star rating,100 3rd St S,154,American (Traditional); Gastropubs\n9. Piggy’s Restaurant,4 star rating,501 Front St S,131,Steakhouses; Seafood; Sandwiches\n10. Schuby’s Neighborhood Butcher,4.5 star rating,321 State St,17,Butcher; Delis; Caterers'

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

In [51]:
webpage=re.compile(r'pagination-link')
link = (restaurants
           >> find_all('a',attrs={'class':webpage})
           >> map(tlz.get('href'))
           )
link

['/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=10',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=20',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=30',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=40',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=50',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=60',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=70',
 '/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=80']

In [73]:
https = [line.replace("/","https://www.yelp.com/") for line in link]
https

['https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=10',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=20',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=30',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=40',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=50',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=60',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=70',
 'https://www.yelp.com/search?cflt=restaurants&find_loc=LaCrosse%2C%20Wi&start=80']

In [65]:
first_10 = web_scrape('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=0')

In [66]:
joined = [",".join(line) for line in first_10]
joined

['1. The Waterfront Restaurant & Tavern,4.5 star rating,328 Front St S,226,American (New); Seafood; Steakhouses',
 '2. Buzzard Billy’s,4 star rating,222 Pearl St,276,American (Traditional); Cajun/Creole',
 '3. Restore Public House,4.5 star rating,1810 State St,17,American (Traditional)',
 '4. Lovechild Restaurant,4.5 star rating,300 3rd St S,109,American (New)',
 '5. The Charmant,4.5 star rating,101 State St,149,French; Cocktail Bars',
 '6. The Freighthouse Restaurant,4 star rating,107 Vine St,125,Seafood; Steakhouses; Desserts',
 '7. Howie’s on La Crosse,4 star rating,1128 La Crosse St,57,American (New); Pubs',
 '8. The Crow,3.5 star rating,100 3rd St S,154,American (Traditional); Gastropubs',
 '9. Piggy’s Restaurant,4 star rating,501 Front St S,131,Steakhouses; Seafood; Sandwiches',
 '10. Schuby’s Neighborhood Butcher,4.5 star rating,321 State St,17,Butcher; Delis; Caterers']

In [68]:
first_10 = '\n'.join(joined)
first_10

'1. The Waterfront Restaurant & Tavern,4.5 star rating,328 Front St S,226,American (New); Seafood; Steakhouses\n2. Buzzard Billy’s,4 star rating,222 Pearl St,276,American (Traditional); Cajun/Creole\n3. Restore Public House,4.5 star rating,1810 State St,17,American (Traditional)\n4. Lovechild Restaurant,4.5 star rating,300 3rd St S,109,American (New)\n5. The Charmant,4.5 star rating,101 State St,149,French; Cocktail Bars\n6. The Freighthouse Restaurant,4 star rating,107 Vine St,125,Seafood; Steakhouses; Desserts\n7. Howie’s on La Crosse,4 star rating,1128 La Crosse St,57,American (New); Pubs\n8. The Crow,3.5 star rating,100 3rd St S,154,American (Traditional); Gastropubs\n9. Piggy’s Restaurant,4 star rating,501 Front St S,131,Steakhouses; Seafood; Sandwiches\n10. Schuby’s Neighborhood Butcher,4.5 star rating,321 State St,17,Butcher; Delis; Caterers'

In [78]:
the_rest = [web_scrape(page) for page in https]
the_rest

['11. Digger’s Sting Restaurant,4 star rating,122 3rd St N,62,American (New); Steakhouses\n12. 4 Sisters Wine Bar and Tapas,3.5 star rating,100 Harborview Plz,111,Wine Bars; Tapas/Small Plates; Tapas Bars\n13. Milwaukee Burger Company,3.5 star rating,3039 Medco Ct,23,Burgers; Sports Bars; Beer Bar\n14. Uno Venti Pizzeria,4 star rating,120 King St,17,Pizza; Italian; Beer Bar\n15. Five Star Eggrolls,3.5 star rating,1203 La Crosse St,9,Thai; Laotian; Chinese\n16. River Rats Bar and Grill,4.5 star rating,1311 La Crescent Pl,28,American (New); Burgers; Cocktail Bars\n17. Hmong’s Golden Egg Roll,4 star rating,901 State St,69,Laotian; Vietnamese; Thai\n18. Señor Villa,4 star rating,325 West Ave N,69,Mexican\n19. Burritos House,4.5 star rating,1205 La Crosse St,30,Mexican\n20. Le Chateau,4.5 star rating,410 Cass St,51,French; Wine Bars; Cocktail Bars',
 '21. Iguana’s Mexican Street Café,4 star rating,1800 State St,71,Mexican\n22. Hungry Peddler,4 star rating,3429 Mormon Coulee Rd,64,American (

In [81]:
the_Rest = "\n".join(the_rest)
the_Rest

'11. Digger’s Sting Restaurant,4 star rating,122 3rd St N,62,American (New); Steakhouses\n12. 4 Sisters Wine Bar and Tapas,3.5 star rating,100 Harborview Plz,111,Wine Bars; Tapas/Small Plates; Tapas Bars\n13. Milwaukee Burger Company,3.5 star rating,3039 Medco Ct,23,Burgers; Sports Bars; Beer Bar\n14. Uno Venti Pizzeria,4 star rating,120 King St,17,Pizza; Italian; Beer Bar\n15. Five Star Eggrolls,3.5 star rating,1203 La Crosse St,9,Thai; Laotian; Chinese\n16. River Rats Bar and Grill,4.5 star rating,1311 La Crescent Pl,28,American (New); Burgers; Cocktail Bars\n17. Hmong’s Golden Egg Roll,4 star rating,901 State St,69,Laotian; Vietnamese; Thai\n18. Señor Villa,4 star rating,325 West Ave N,69,Mexican\n19. Burritos House,4.5 star rating,1205 La Crosse St,30,Mexican\n20. Le Chateau,4.5 star rating,410 Cass St,51,French; Wine Bars; Cocktail Bars\n21. Iguana’s Mexican Street Café,4 star rating,1800 State St,71,Mexican\n22. Hungry Peddler,4 star rating,3429 Mormon Coulee Rd,64,American (Trad

In [69]:
with open('HW_5.csv','w') as outfile:
    outfile.write(first_10)

In [83]:
with open('HW_5.csv','a') as outfile:
    outfile.write(the_Rest)

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.